## Setup

Note - I couldn't figure out how to fix local imports for a while. Solution ended up being to make sure that this version of `transformer_lens` is before my libraries in `sys.path` (hence I'm inserting it at position zero in the code below).

In [ ]:
import os, sys
from pathlib import Path
p = Path(r"C:\Users\calsm\Documents\AI Alignment\SERIMATS_23\seri_mats_23_streamlit_pages")
if os.path.exists(str_p := str(p.resolve())):
    os.chdir(str_p)
    if (sys.path[0] != str_p):
        sys.path.insert(0, str_p)

from transformer_lens.cautils.notebook import *
import gzip

from transformer_lens.rs.callum2.cspa.cspa_functions import (
    FUNCTION_STR_TOKS,
    # project,
    # get_effective_embedding,
    get_cspa_results,
    get_cspa_results_batched,
    # first_occurrence_2d,
)
from transformer_lens.rs.callum2.cspa.cspa_utils import (
    parse_str,
    parse_str_toks_for_printing,
)
from transformer_lens.rs.callum2.cspa.cspa_plots import (
    generate_scatter,
    generate_loss_based_scatter,
    add_cspa_to_streamlit_page,
)
from transformer_lens.rs.callum2.generate_st_html.utils import (
    ST_HTML_PATH,
    parse_str_tok_for_printing,
)
from transformer_lens.rs.callum2.generate_st_html.model_results import (
    get_model_results,
)
from transformer_lens.rs.callum2.generate_st_html.generate_html_funcs import (
    # generate_html_for_cspa_plots,
    # generate_html_for_logit_plot,
    # generate_html_for_DLA_plot,
    generate_4_html_plots,
    CSS,
)
from transformer_lens.rs.callum2.cspa.cspa_semantic_similarity import (
    get_equivalency_toks,
    get_related_words,
    concat_lists,
    # is_token,
    # get_list_with_no_repetitions,
    make_list_correct_length,
    create_full_semantic_similarity_dict,
)

clear_output()

In [ ]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device="cuda" # "cuda"
    # fold value bias?
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

clear_output()

In [ ]:
BATCH_SIZE = 1000 # 80 for viz?
SEQ_LEN = 61 # 70 for viz (no more, because attn)
batch_idx = 36

NEGATIVE_HEADS = [(10, 7), (11, 10)]

def process_webtext(
    seed: int = 6,
    batch_size: int = BATCH_SIZE,
    indices: Optional[List[int]] = None,
    seq_len: int = SEQ_LEN,
    verbose: bool = False,
):
    DATA_STR = get_webtext(seed=seed)
    if indices is None:
        DATA_STR = DATA_STR[:batch_size]
    else:
        DATA_STR = [DATA_STR[i] for i in indices]
    DATA_STR = [parse_str(s) for s in DATA_STR]

    DATA_TOKS = model.to_tokens(DATA_STR)
    DATA_STR_TOKS = model.to_str_tokens(DATA_STR)

    if seq_len < 1024:
        DATA_TOKS = DATA_TOKS[:, :seq_len]
        DATA_STR_TOKS = [str_toks[:seq_len] for str_toks in DATA_STR_TOKS]

    DATA_STR_TOKS_PARSED = list(map(parse_str_toks_for_printing, DATA_STR_TOKS))

    clear_output()
    if verbose:
        print(f"Shape = {DATA_TOKS.shape}\n")
        print("First prompt:\n" + "".join(DATA_STR_TOKS[0]))

    return DATA_TOKS, DATA_STR_TOKS_PARSED


DATA_TOKS, DATA_STR_TOKS_PARSED = process_webtext(verbose=True)
DATA_TOKS_MINI = DATA_TOKS[[32, 36], :60]
DATA_STR_TOKS_PARSED_MINI = [DATA_STR_TOKS_PARSED[32][:60], DATA_STR_TOKS_PARSED[36][:60]]

# DATA_TOKS_SMALL, DATA_STR_TOKS_PARSED_SMALL = process_webtext(indices=list(range(32, 42)), verbose=True) # 
# BATCH_SIZE, SEQ_LEN = DATA_TOKS.shape

# MINIBATCH_SIZE = BATCH_SIZE // 3
# DATA_TOKS_1 = DATA_TOKS[:MINIBATCH_SIZE]
# DATA_TOKS_2 = DATA_TOKS[MINIBATCH_SIZE:2*MINIBATCH_SIZE]
# DATA_TOKS_3 = DATA_TOKS[2*MINIBATCH_SIZE:]
# DATA_STR_TOKS_PARSED_1 = DATA_STR_TOKS_PARSED[:MINIBATCH_SIZE]
# DATA_STR_TOKS_PARSED_2 = DATA_STR_TOKS_PARSED[MINIBATCH_SIZE:2*MINIBATCH_SIZE]
# DATA_STR_TOKS_PARSED_3 = DATA_STR_TOKS_PARSED[2*MINIBATCH_SIZE:]

## Hardcoded semantic similarity

This will take tokens, and return the tokens of semantically similar words. 

There are 3 categories of semantically similar tokens `s*` for any given token `s`:

1. Equivalence relations - this captures things like plurals, tokenization, capitalization.
2. Superstrings - for instance, of you have `"keley"` this gives you `" Berkeley"`.
3. Substrings - for instance, of you have `" Berkeley"` this gives you `"keley"`.

How does this work?

1. For each token, generate all (1), and then see which ones actually split into multiple tokens (these become (3)).
2. Iterate through this entire dict to generate (2)s for every token (this is basically like flipping the arrows in the other direction).

### Problems with this method

There are 4 problems with this method. I think (1) and (3) are the most problematic.

1. The pluralization isn't sufficiently flexible, and it'll miss out on categories of things, for example:
    * `" write"` and `" writing"` and `" writer"`
    * `" rental"` and `" rented"` and `" renting"`
    * (OV and QK circuits show that these do suppress each other)
    * Possible solution - more hardcoded rules?
2. Misses some important things which aren't semantically similar as we've defined it, e.g. `1984` and `1985` aren't semantically similar (OV and QK circuits show that they do suppress each other)
    * Possible solution - ???

However, this maybe isn't worth further optimization, because it doesn't marginally improve the results by much.

In [ ]:
from pattern.text.en import conjugate, PRESENT, PAST, FUTURE, SUBJUNCTIVE, INFINITIVE, PROGRESSIVE, PLURAL, SINGULAR
from nltk.stem import WordNetLemmatizer
import nltk
MY_TENSES = [PRESENT, PAST, FUTURE, SUBJUNCTIVE, INFINITIVE, PROGRESSIVE]
MY_NUMBERS = [PLURAL, SINGULAR]
from nltk.corpus import wordnet
nltk.download('wordnet')

In [ ]:
# cspa_semantic_dict_reversed, cspa_semantic_dict = create_full_semantic_similarity_dict(model, full_version=True)

# with open(ST_HTML_PATH.parent.parent / "cspa/cspa_semantic_dict_full.pkl", "wb") as f:
#     pickle.dump(cspa_semantic_dict, f)

cspa_semantic_dict = pickle.load(open(ST_HTML_PATH.parent.parent / "cspa/cspa_semantic_dict_full.pkl", "rb"))

In [ ]:
display(HTML("<h2>Related words</h2>"))

for word in ["Berkeley", "pier", "pie", "ring", "device", "robot", "w"]:
    print(get_related_words(word, model))

In [ ]:
display(HTML("<h2>Equivalency tokens</h2>"))

for tok in [" Berkeley", " Pier", " pier", "pie", " pies", " ring", " device", " robot", "w"]:
    print(f"{tok!r:>10} -> {get_equivalency_toks(tok, model)}")

In [ ]:
table = Table("Source token", "Top 3 related", "All semantically related", title="Semantic similarity: bidirectional, superstrings, substrings")

str_toks = [" Berkeley", "keley", " University", " Mary", " Pier", " pier", "NY", " ring", " W", " device", " robot", " jump", " driver", " Cairo"]
print_cutoff = 70
def cutoff(s):
    if len(s_str := str(s)) >= print_cutoff: return s_str[:print_cutoff-4] + ' ...'
    else: return s_str

for str_tok in str_toks:
    top3_sim = "\n".join(list(map(repr, concat_lists(cspa_semantic_dict[str_tok])[:3])))
    bidir, superstr, substr = cspa_semantic_dict[str_tok]
    all_sim = "\n".join([
        cutoff(f"{len(bidir)} Bidir: {bidir}"),
        cutoff(f"{len(superstr)} Super: {superstr}"),
        cutoff(f"{len(substr)} Sub:   {substr}"),
    ]) + "\n"
    table.add_row(repr(str_tok), top3_sim, all_sim)

rprint(table)

## Running CSPA code

In [ ]:
# This didn't preserve the BOS stuff

cspa_results_1, s_sstar_pairs_1 = get_cspa_results_batched(
    model = model,
    toks = DATA_TOKS, # [:50],
    max_batch_size = 100, # 50,
    negative_head = (10, 7),
    components_to_project = ["o"],
    K_unembeddings = 8, # None
    K_semantic = 1, # 3
    semantic_dict = cspa_semantic_dict,
    effective_embedding = "W_E (including MLPs)",
    use_cuda = True,
    verbose = True,
    return_dla = True,
    start_idx = 0,
)
# TODO - speed of projection onto large number of vectors is the main bottleneck by far; try to speed it up

fig_dict = generate_scatter(cspa_results_1, DATA_STR_TOKS_PARSED, batch_index_colors_to_highlight=[51, 300])
fig_loss_line = generate_loss_based_scatter(cspa_results_1, nbins=200)

# ! Save figures for ST page
# fig_dict["loss-based scatter"] = fig_loss_line
# p = "/home/ubuntu/SERI-MATS-2023-Streamlit-pages/transformer_lens/rs/callum2/st_page/media"
# with gzip.open(f"{p}/CSPA_figs.pkl", "wb") as f:
#     pickle.dump(fig_dict, f)

In [ ]:
# This didn't preserve the BOS stuff

cspa_results, s_sstar_pairs = get_cspa_results_batched(
    model = model,
    toks = DATA_TOKS, # [:50],
    max_batch_size = 100, # 50,
    negative_head = (10, 7),
    components_to_project = ["o"],
    K_unembeddings = 8, # None
    K_semantic = 5, # 3
    semantic_dict = cspa_semantic_dict,
    effective_embedding = "W_E (including MLPs)",
    use_cuda = True,
    verbose = True,
    return_dla = True,
    start_idx = 0,
)
# TODO - speed of projection onto large number of vectors is the main bottleneck by far; try to speed it up

fig_dict = generate_scatter(cspa_results, DATA_STR_TOKS_PARSED, batch_index_colors_to_highlight=[51, 300])
fig_loss_line = generate_loss_based_scatter(cspa_results, nbins=200)

# ! Save figures for ST page
fig_dict["loss-based scatter"] = fig_loss_line
p = "/root/SERI-MATS-2023-Streamlit-pages/transformer_lens/rs/callum2/st_page/media"
with gzip.open(f"{p}/CSPA_figs.pkl", "wb") as f:
    pickle.dump(fig_dict, f)

# Adding CSPA to the Streamlit page ("Browse Examples")

This code adds the CSPA plots to the HTML plots for the Streamlit page. It creates a 5th tab called `CSPA`, and adds to the logit and DLA plots in the second tab (the latter is mainly for our use, while we're iterating on and improving the CSPA code).

I've added to this code in a pretty janky way, so that it can show more than one CSPA plot stacked on top of each other.

In [ ]:
add_cspa_to_streamlit_page(
    cspa_results = {"$K_{sem}=4$": cspa_results, "$K_{sem}=1$": cspa_results_1},
    s_sstar_pairs = {"$K_{sem}=4$": s_sstar_pairs, "$K_{sem}=1$": s_sstar_pairs_1},
    html_plots_filename = f"GZIP_HTML_PLOTS_b301_s61.pkl",
    data_str_toks_parsed = DATA_STR_TOKS_PARSED,
    toks_for_doing_DLA = DATA_TOKS,
    model = model,
    verbose = True,
    test_idx = 40,
)

In [ ]:
b = 51
add_cspa_to_streamlit_page(
    cspa_results = cspa_results,
    s_sstar_pairs = s_sstar_pairs,
    html_plots_filename = f"GZIP_HTML_PLOTS_b{b}_s61.pkl",
    data_str_toks_parsed = DATA_STR_TOKS_PARSED,
    toks_for_doing_DLA = DATA_TOKS,
    model = model,
    verbose = True,
    # test_idx = 32,
)

b = 200
add_cspa_to_streamlit_page(
    cspa_results = {"k=4": cspa_results, "k=1": cspa_results_1},
    s_sstar_pairs = {"k=4": s_sstar_pairs, "k=1": s_sstar_pairs_1},
    html_plots_filename = f"GZIP_HTML_PLOTS_b{b}_s61.pkl",
    data_str_toks_parsed = DATA_STR_TOKS_PARSED,
    toks_for_doing_DLA = DATA_TOKS,
    model = model,
    verbose = True,
    # test_idx = 32,
)

In [ ]:
def cos_sim_of_toks(
    toks1: List[str],
    toks2: List[str],
):
    U1 = model.W_U.T[model.to_tokens(toks1, prepend_bos=False).squeeze()]
    U2 = model.W_U.T[model.to_tokens(toks2, prepend_bos=False).squeeze()]

    if U1.ndim == 1: U1 = U1.unsqueeze(0)
    if U2.ndim == 1: U2 = U2.unsqueeze(0)

    U1_normed = U1 / t.norm(U1, dim=-1, keepdim=True)
    U2_normed = U2 / t.norm(U2, dim=-1, keepdim=True)

    imshow(
        U1_normed @ U2_normed.T,
        title = "Cosine similarity of unembeddings",
        x = toks2,
        y = toks1,
    )

cos_sim_of_toks(
    [" stuff"],
    [" devices", " phones", " screens", " device", " phone", " Android"]
)

# Quick code to get mean ablations for the streamlit page

In [ ]:
DATA_TOKS, DATA_STR_TOKS_PARSED = process_webtext(verbose=False, batch_size=1000, seq_len=100)
assert DATA_TOKS.shape == (1000, 100)

In [ ]:
result_all = t.empty((2, 0, 100, model.cfg.d_model))

for i in range(0, 1000, 100):
    _, cache = model.run_with_cache(
        DATA_TOKS[i: i+100],
        return_type=None,
        names_filter=lambda name: name in [utils.get_act_name("result", layer) for layer in [10, 11]]
    )
    result_L10H7 = cache["result", 10][:, :, 7].cpu() # [batch seq d_model]
    result_L11H10 = cache["result", 11][:, :, 10].cpu() # [batch seq d_model]
    result_all = t.cat([
        result_all, 
        t.stack([result_L10H7, result_L11H10])
    ], dim=1)

result_mean = result_all.mean(dim=1)
assert result_mean.shape == (2, 100, model.cfg.d_model)

t.save(result_mean, f"/home/ubuntu/SERI-MATS-2023-Streamlit-pages/transformer_lens/rs/callum2/st_page/media/result_mean.pt")

# Testing the code for "love and war"

In [ ]:
result_mean_as_tensor = t.load(ST_HTML_PATH / "result_mean.pt")
result_mean = {(10, 7): result_mean_as_tensor[0], (11, 10): result_mean_as_tensor[1]}

prompt = "I picked up the first box. I picked up the second box. I picked up the third and final box."
toks = model.to_tokens(prompt)
str_toks = model.to_str_tokens(toks)
if isinstance(str_toks[0], str): str_toks = [str_toks]
# Parse the string tokens for printing
str_toks_parsed = [list(map(parse_str_tok_for_printing, s)) for s in str_toks]

model_results = get_model_results(
    model,
    toks=toks,
    negative_heads=[(10, 7), (11, 10)],
    result_mean=result_mean,
    verbose=False
)
HTML_PLOTS_NEW = generate_4_html_plots(
    model=model,
    data_toks=toks,
    data_str_toks_parsed=str_toks_parsed,
    negative_heads=[(10, 7), (11, 10)],
    model_results=model_results,
    save_files=False,
)
cspa_results, s_sstar_pairs = get_cspa_results(
    model=model,
    toks=toks,
    negative_head=(10, 7), #  this currently doesn't do anything; it's always 10.7
    components_to_project=["o"],
    K_unembeddings=5,
    K_semantic=3,
    semantic_dict=cspa_semantic_dict,
    effective_embedding="W_E (including MLPs)",
    result_mean=result_mean,
    use_cuda=False,
    return_dla=True,
)
HTML_PLOTS_NEW = add_cspa_to_streamlit_page(
    cspa_results=cspa_results,
    s_sstar_pairs=s_sstar_pairs,
    data_str_toks_parsed=str_toks_parsed,
    model=model,
    HTML_PLOTS=HTML_PLOTS_NEW,
    toks_for_doing_DLA=toks,
    verbose=False,
    test_idx=0,
)